In [1]:
import numpy as np
import astropy.io.fits as pf
import astropy.table as aptbl
import re
import matplotlib.pyplot as plt
import astropy
import astropy.table as aptbl
import astropy.units as u
from astropy.table import QTable
import os
from astropy.io import ascii
from astropy.io.ascii import masked
import gzip
#dles = u.dimensionless_unscaled
Land = np.logical_and
Lor = np.logical_or
Lnot = np.logical_not
from functools import reduce

In [2]:
file = pf.open("GaiaSearch")
parallax = file[1].data["parallax"] 
parallax_over_error = file[1].data["parallax_over_error"]
Vs = np.asarray((file[1].data["Vra"], file[1].data["Vdec"])).T
Num = file[1].data["Num"]
P = file[1].data["Period_ms"]
Separation = file[1].data["Separation"]
#print(file[1].data.keys())

In [3]:
def nan_float(x):
    try:
        y = float(x)
    except ValueError:
        y = np.nan
    return y

In [4]:
P = np.asarray([ nan_float(t) for t in P ])

In [5]:
Numset = set(Num)

In [6]:
cutchisqr = 9 #3 sigma

In [7]:
def calc_cut(Vs):
    medianvs = np.median(Vs, axis = 0)
    deviations = Vs-medianvs.reshape((1,2))
    
    stdmag = np.mean((deviations)**2, axis = 0)
    covxx = stdmag[0]
    covyy = stdmag[1]
    covxy = np.mean(deviations[:,0]*deviations[:,1],axis = 0)
    cov = np.array([[covxx,covxy],[covxy,covyy]])
    E = np.linalg.inv(cov)
    chisqr = np.einsum("ni,ij,nj->n",deviations, E, deviations)
    #Vmags = np.hypot(Vs[:,0]-medianvs[0],Vs[:,1]-medianvs[1])
    ind = chisqr<=cutchisqr
    V_keep = Vs[ind]
    return(V_keep)

In [8]:
LSRs = {}
#LSRlist = []
for n in Numset:
    Vn = Vs[Land(Num == n, parallax_over_error >= 3.0)]
    oldlen = len(Vn)+1
    while oldlen > len(Vn):
        oldlen = len(Vn)
        Vn = calc_cut(Vn)
    
    vlsr = np.median(Vn, axis = 0)
    deviations = Vn-vlsr.reshape((1,2))
    stdmag = np.mean((deviations)**2, axis = 0)
    covxx = stdmag[0]
    covyy = stdmag[1]
    covxy = np.mean(deviations[:,0]*deviations[:,1],axis = 0)
    LSRs[n] = (vlsr,[covxx,covyy,covxy],len(Vn))#(VraLSR, VdecLSR)
    

In [10]:
#LSRlist = np.array([["Num","Vra","Vdec","Sigmaxx","Sigmayy","Sigmaxy","Length"],[Num,vlsr,vlsr,np.sqrt(covxx),np.sqrt(covyy),np.sqrt(covxy),len(Vn)]])
cutchisqr_1 = 9

In [11]:
outputdata = []
#for coldef in file[1].data.columns:
    #cn = coldef.name
    #col = file[1].data.columns[cn]
    #outcol = []
names = ["Num", "Type", "Pipeline", "Period_ms", "RaJ2000_hhmmss", "DecJ2000_ddmmss", "Binary", 
         "datedetection", "col10", "col11", "col12", "col13", "col14", "col15", "col16", "col17", 
         "Date_of_discovery", "NOTE", "col20", "col21", "col22", "col23", "DM", "GAL_LONG", "GAL_LAT", "Distance_DM", 
         "source_id", "ref_epoch", "ra", "ra_error", "dec", "dec_error", "parallax", "parallax_error", "pmra", 
         "pmra_error", "pmdec", "pmdec_error", "ra_dec_corr", "ra_parallax_corr", "ra_pmra_corr", "ra_pmdec_corr", 
         "dec_parallax_corr", "dec_pmra_corr", "dec_pmdec_corr", "parallax_pmra_corr", "parallax_pmdec_corr", 
         "pmra_pmdec_corr", "dr2_radial_velocity", "dr2_radial_velocity_error", "dr2_rv_template_teff", 
         "dr2_rv_template_logg", "dr2_rv_template_fe_h", "Separation", "Vra", "Vdec"]
#units = [dles, dles, dles, u.ms, u.hourangle, u.deg, dles, dles, dles
#                                                            , dles, dles, dles, dles, dles, dles, dles, dles, dles, dles, 
#                                                           dles, dles, dles, dles, u.deg, u.deg, u.pc, dles, u.yr, u.deg,u.mas,
#                                                           u.deg, u.mas, u.mas, u.mas, u.mas/u.yr, u.mas/u.yr, u.mas/u.yr,
#                                                           u.mas/u.yr, dles, dles, dles, dles, dles, dles, dles, dles, 
#                                                            dles, dles, u.km/u.s, u.km/u.s, u.K, u.dex(u.cm/u.s**2), 
#                                                            u.dex, u.deg, u.km/u.s, u.km/u.s]
units = {"GAL_LONG": u.deg, "GAL_LAT": u.deg, "ra": u.deg, "dec": u.deg, 
         "Separation": u.deg, "Distance_DM": u.pc, "ref_epoch": u.yr, "ra_error": u.mas, "dec_error": u.mas, 
         "parallax": u.mas, "parallax_error": u.mas, "pmra": u.mas/u.yr, "pmra_error": u.mas/u.yr, "pmdec": u.mas/u.yr, 
         "pmdec_error": u.mas/u.yr, "dr2_radial_velocity": u.km/u.s, "dr2_radial_velocity_error": u.km/u.s, 
        "Vra": u.km/u.s, "Vdec": u.km/u.s, "dr2_rv_template_teff": u.K, "dr2_rv_template_logg": u.dex(u.cm/u.s**2), 
         "dr2_rv_template_fe_h": u.dex}
for cn in names:
    #cn = coldef.name
    col = file[1].data[cn]
    outcol = []
    for n in Numset:
        Numind = Land(Num == n, parallax_over_error >= 3.0)
        Pn = P[Numind]
        if np.any(Pn <= 1e3):
           maxSep = 3.0 / 60.0
        elif np.any(Pn > 1e3):
           maxSep = 5.0 / 60.0
        else:
           maxSep = 15.0/60.0
        Numind = Land(Numind, Separation<=maxSep)
        Vn = Vs[Numind]
        vlsr,covs,N = LSRs[n]
        deviations = Vn-vlsr.reshape((1,2))
        covxx,covyy,covxy = covs
        cov = np.array([[covxx,covxy],[covxy,covyy]])
        E = np.linalg.inv(cov)
        chisqr = np.einsum("ni,ij,nj->n",deviations, E, deviations)
        ind = chisqr>=cutchisqr_1
        #outputdata.append(np.asarray(file[1].data[Numind][ind]))
        #outcol.extend(col[Numind][ind])
        outcol.extend(col[Numind][ind])
    outputdata.append(outcol)

outputdata = aptbl.QTable(outputdata, names=names,units = units)

In [12]:
class gzwrapper:
    def __init__(self, gzfile):
        self.gzfile=gzfile
    def write(self, line):
        return self.gzfile.write(line.encode())

In [13]:
#olines = ascii.write(outputdata, format='ipac')
#print(olines[0:3])
#print(olines)
#content = b outputdata
#print(type(olines), dir(olines))
#olines = ascii.write(outputdata, output = None, format='ipac')
#for line in olines:
#    f.write(line)
with gzip.open('/Users/tomluan/Desktop/ljm/FAST Intern/outputdata.dat.gz', 'w') as f:
#    olines = ascii.write(outputdata, output = None, format='ipac')
#    for line in olines:
#        f.write(line)
#    ascii.write(outputdata, output = f, format='ipac')
    fwrap = gzwrapper(f)
    ascii.write(outputdata, output=fwrap, format="ipac")

In [14]:
outputdata["name"] = outputdata["source_id"]
with gzip.open('/Users/tomluan/Desktop/ljm/FAST Intern/gaiatargetlist.dat.gz', 'w') as f:
    fwrap = gzwrapper(f)
    ascii.write(outputdata, output=fwrap, include_names = ["name", "ra", "dec", "parallax"], format="ipac")